In [2]:
import sys
import os
import time
import logging
import datetime
from datetime import datetime, timedelta
import yaml
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import requests
import pandas_gbq
from dreams_core.googlecloud import GoogleCloud as dgc
from dreams_core import core as dc
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

# load dotenv
load_dotenv()


# import local files if necessary
sys.path.append('..//src')
from utils import load_config, cw_filter_df
import training_data as td
importlib.reload(td)
import feature_engineering as fe
importlib.reload(fe)
import coin_wallet_metrics as cwm
importlib.reload(cwm)


# configure logger
logger = dc.setup_logger()
logger.setLevel(logging.INFO)

# Custom format function for displaying numbers
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

#### Load the datasets

In [3]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe) 

config = load_config('../config/config.yaml')
metrics_config = load_config('../config/config_metrics.yaml')

# retrieve and clean prices data
prices_df = td.retrieve_prices_data()
prices_df,_ = td.fill_prices_gaps(prices_df,config['data_cleaning']['max_gap_days'])
logger.info(f"Prices data shape: {prices_df.shape}")

# retrieve transfers data
transfers_df = td.retrieve_transfers_data(
    config['training_data']['training_period_start'],
    config['training_data']['modeling_period_start'],
    config['training_data']['modeling_period_end']
    )

# compile profits_df
profits_df = td.prepare_profits_data(transfers_df, prices_df)
profits_df = td.calculate_wallet_profitability(profits_df)
profits_df,_ = td.clean_profits_df(profits_df, config['data_cleaning'])

# identify sharks
shark_coins_df = td.classify_shark_coins(profits_df, config['training_data'])
shark_wallets_df = td.classify_shark_wallets(shark_coins_df,config['training_data'])

# generate and flatten buysell_metrics
cohort_wallets = shark_wallets_df[shark_wallets_df['is_shark']==True]['wallet_address'].unique()
cohort_coins = shark_coins_df['coin_id'].unique()
buysell_metrics_df = cwm.generate_buysell_metrics_df(profits_df,config['training_data']['training_period_end'],cohort_wallets,cohort_coins)
flattened_buysell_metrics_df = fe.flatten_coin_date_df(buysell_metrics_df,metrics_config,config['training_data']['training_period_end'])


[12/Sep/2024 23:04:29] INFO [dreams_core.core.retrieve_prices_data:42] retrieved prices data with shape (120763, 3)
[12/Sep/2024 23:04:29] INFO [dreams_core.core.fill_prices_gaps:126] 382 coins had no gaps, 19 coins had gaps filled, and 38 coins were dropped due to large gaps.
[12/Sep/2024 23:04:29] INFO [dreams_core.core.<module>:11] Prices data shape: (110929, 3)
[12/Sep/2024 23:04:58] INFO [dreams_core.core.retrieve_transfers_data:414] retrieved transfers_df with shape (23823401, 5) after 28.1 seconds.
[12/Sep/2024 23:04:58] INFO [dreams_core.core.prepare_profits_data:455] Preparing profits_df data...
[12/Sep/2024 23:05:49] INFO [dreams_core.core.calculate_wallet_profitability:635] Generated profits df after 20.17 seconds
[12/Sep/2024 23:06:12] INFO [dreams_core.core.clean_profits_df:706] Finished cleaning profits_df after 22.70 seconds.
[12/Sep/2024 23:06:21] INFO [dreams_core.core.classify_shark_coins:772] creation of shark_coins_df complete.
[12/Sep/2024 23:06:21] INFO [dreams_co

In [4]:
version='0.0'
modeling_period_start = config['training_data']['modeling_period_start']
output_path = '..//modeling/outputs/flattened_outputs'

# def save_flattened_outputs(flattened_df, output_path, training_period_start, version=None):
# Call flatten_coin_date_df to get the transformed data
flattened_df = flattened_buysell_metrics_df

# Generate timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M')

# Define filename, optionally include version
version_suffix = f"_v{version}" if version else ""
filename = f"flattened_output_{timestamp}_trainingstart_{training_period_start}{version_suffix}.csv"

# Define full path and save the DataFrame
full_path = os.path.join(output_path, filename)
flattened_df.to_csv(full_path, index=False)
